In [1]:
import os
import SimpleITK as sitk
import numpy as np
from tqdm import tqdm

EAT_folder = "/storage/awias/NLDL_Winterschool/predictions_periseg_postprocessed_EAT"

Check minimum size of image if I crop to 1mm resolution

In [2]:
import os
import SimpleITK as sitk
import numpy as np
from tqdm import tqdm

EAT_folder = "/storage/awias/NLDL_Winterschool/predictions_periseg_postprocessed_EAT"

max_voxel_dims = [0, 0, 0]  # x, y, z in voxels

for filename in tqdm(os.listdir(EAT_folder)):
    if not filename.endswith(".nii.gz"):
        continue

    path = os.path.join(EAT_folder, filename)
    img_sitk = sitk.ReadImage(path)
    img = sitk.GetArrayFromImage(img_sitk)
    spacing = img_sitk.GetSpacing()  # (x_spacing, y_spacing, z_spacing)
    
    # Bounding box of nonzero mask
    nonzero = np.nonzero(img)
    z_min, y_min, x_min = np.min(nonzero[0]), np.min(nonzero[1]), np.min(nonzero[2])
    z_max, y_max, x_max = np.max(nonzero[0]), np.max(nonzero[1]), np.max(nonzero[2])
    
    # Size in mm along each axis
    size_mm = [
        (x_max - x_min + 1) * spacing[0],
        (y_max - y_min + 1) * spacing[1],
        (z_max - z_min + 1) * spacing[2],
    ]
    
    # Convert to target spacing 0.5 mm
    size_voxels = [int(np.ceil(s / 0.5)) for s in size_mm]

    # Update maximum over all images
    max_voxel_dims = [max(max_voxel_dims[i], size_voxels[i]) for i in range(3)]

print("Estimated maximal 3D size at 0.5 mm spacing (X, Y, Z):", max_voxel_dims)

100%|██████████| 11/11 [00:04<00:00,  2.75it/s]

Estimated maximal 3D size at 0.5 mm spacing (X, Y, Z): [302, 292, 250]


In [4]:
import os
import SimpleITK as sitk
import numpy as np
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm

EAT_folder = "/storage/awias/NLDL_Winterschool/predictions_periseg_postprocessed_EAT"
target_spacing = 0.5  # mm
n_workers = 8  # Number of parallel processes

def get_max_voxel_dims(filename):
    if not filename.endswith(".nii.gz"):
        return [0, 0, 0]

    path = os.path.join(EAT_folder, filename)
    img_sitk = sitk.ReadImage(path)
    img = sitk.GetArrayFromImage(img_sitk)
    spacing = img_sitk.GetSpacing()  # (x_spacing, y_spacing, z_spacing)

    # Bounding box of nonzero mask
    nonzero = np.nonzero(img)
    if len(nonzero[0]) == 0:
        return [0, 0, 0]  # empty mask

    z_min, y_min, x_min = np.min(nonzero[0]), np.min(nonzero[1]), np.min(nonzero[2])
    z_max, y_max, x_max = np.max(nonzero[0]), np.max(nonzero[1]), np.max(nonzero[2])

    # Size in mm
    size_mm = [
        (x_max - x_min + 1) * spacing[0],
        (y_max - y_min + 1) * spacing[1],
        (z_max - z_min + 1) * spacing[2],
    ]

    # Convert to target spacing
    size_voxels = [int(np.ceil(s / target_spacing)) for s in size_mm]
    return size_voxels

# Get all filenames
file_names = [f for f in os.listdir(EAT_folder) if f.endswith(".nii.gz")]

# Parallel processing
max_voxel_dims = [0, 0, 0]
with ProcessPoolExecutor(max_workers=n_workers) as executor:
    futures = {executor.submit(get_max_voxel_dims, f): f for f in file_names}
    for future in tqdm(as_completed(futures), total=len(futures)):
        dims = future.result()
        max_voxel_dims = [max(max_voxel_dims[i], dims[i]) for i in range(3)]

print("Estimated maximal 3D size at 0.5 mm spacing (X, Y, Z):", max_voxel_dims)

  0%|          | 0/11 [00:00<?, ?it/s]Process ForkServerProcess-9:
Process ForkServerProcess-10:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/awias/miniconda3/envs/standard/lib/python3.14/multiprocessing/process.py", line 320, in _bootstrap
    self.run()
    ~~~~~~~~^^
  File "/home/awias/miniconda3/envs/standard/lib/python3.14/multiprocessing/process.py", line 320, in _bootstrap
    self.run()
    ~~~~~~~~^^
  File "/home/awias/miniconda3/envs/standard/lib/python3.14/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/awias/miniconda3/envs/standard/lib/python3.14/concurrent/futures/process.py", line 242, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/awias/miniconda3/envs/standard/lib/python3.14/multiprocessing/queues.py", line 120, in get
    return _ForkingPickler.loads(res)
           ~~~~~~~~~~~~~~~~~~~~~^^^^^
  Fi

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.